<a href="https://colab.research.google.com/github/xykkong/video-transcription-bert-faiss/blob/main/main.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Pt1. Preparation

## Installing Dependencies

In [ ]:
!pip install SpeechRecognition
!pip install pydub
!pip install transformers
!pip install faiss-cpu
!pip install torch
!pip install pandas
!pip install pocketsphinx

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 32.8/32.8 MB 16.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.6/17.6 MB 36.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 29.2/29.2 MB 11.8 MB/s eta 0:00:00


## Importing libraries

In [ ]:
import speech_recognition as sr
from pydub import AudioSegment
from transformers import BertTokenizer, BertModel
import faiss
import torch
import argparse
import os
import pandas as pd

#Pt2. Extract audio from video

In [ ]:
video_filename = "video1.mp4"
prefix = video_filename.split('.')[0]
audio_filename = f"{prefix}_audio.wav"
video = AudioSegment.from_file(video_filename, format="mp4")
audio = video.set_channels(1).set_frame_rate(16000).set_sample_width(2)
audio.export(audio_filename, format="wav")

<_io.BufferedRandom name='video1_audio.wav'>

#Pt3 Extract vide

In [ ]:
# Initialize recognizer
recognizer = sr.Recognizer()

with sr.AudioFile(audio_filename) as source:
  # read the entire audio file
  audio_text = recognizer.record(source)

transcript = recognizer.recognize_google(audio_text)

# Save the transcript
with open(f"{prefix}_transcript.txt", "w") as file:
  file.write(transcript)


# Pt4. Generate word embeedings

## Split transcription in a list of sentences

In [ ]:
sentences = []
buffer = ''
with open('./video1_transcript.txt', 'r') as file:
  while True:
    chunk = file.read(1024)
    if not chunk:
      break
    text = (buffer + chunk).split('.')
    for sentence in text[:-1]:
      sentences.append(sentence.strip() + ".")
    buffer = text[-1]  # Incomplete sentence, stored for the next iteration

if buffer:  # Process any remaining incomplete sentence at the end of the file
  sentences.append(buffer.strip() + ".")

print(len(sentences))
sentences

29


['The United States of America, commonly known as America, is a country primarily located in North America.',
 'The third-largest country in the world by land and total area, the United States consists of 50 states, a federal district, five major unincorporated territories, nine Minor Outlying Islands and includes 326 Indian reservations.',
 'It shares land borders with Canada to its north and with Mexico to its south and has maritime borders with the Bahamas, Cuba, Russia, and other nations.',
 'With a population of over 334 million, it is the most populous country in the Americas and the third-most populous in the world.',
 'The national capital of the United States is Washington, DC, and its most populous city and principal financial center is New York City.',
 "Ancestors of America's indigenous peoples migrated across the Bering land bridge more than 12,000 years ago.",
 'Beginning in 1607, British colonization led to the establishment of the Thirteen Colonies in what is now the Ea

## Encoding sentences

In [ ]:
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model = BertModel.from_pretrained('bert-base-uncased')

encoding = tokenizer.batch_encode_plus(
    sentences,                 # List of input texts
    padding=True,              # Pad to the maximum sequence length
    truncation=True,           # Truncate to the maximum sequence length if necessary
    return_tensors='pt',       # Return PyTorch tensors
    add_special_tokens=True    # Add special tokens CLS and SEP
)
input_ids = encoding['input_ids']  # Token IDs
attention_mask = encoding['attention_mask']  # Attention mask

print(tokenizer.convert_ids_to_tokens(input_ids[0]))
print(f"Input ID: {input_ids}")
print(f"Attention mask: {attention_mask}")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:72: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

['[CLS]', 'the', 'united', 'states', 'of', 'america', ',', 'commonly', 'known', 'as', 'america', ',', 'is', 'a', 'country', 'primarily', 'located', 'in', 'north', 'america', '.', '[SEP]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]']
Input ID: tensor([[ 101, 1996, 2142,  ...,    0,    0,    0],
        [ 101, 1996, 2353,  ...,    0,    0,    0],
        [ 101, 2009, 6661,  ...,    0,    0,    0],
        ...,
        [ 101, 1996, 2142,  ...,    0,    0,    0],
        [ 101, 2009, 2003,  ...,    0,    0,    0],
        [ 101, 2009, 2003,  ...,    0,    0,    0]])
Attentio

##Generate word embeedings

In [ ]:
# Generate embeddings using BERT model
outputs = None
with torch.no_grad():
  outputs = model(input_ids, attention_mask=attention_mask)
  embeddings = outputs.last_hidden_state.mean(dim=1)  # This contains the embeddings
dimension = embeddings.shape[1]
embeddings

tensor([[-0.1939, -0.0773,  0.0092,  ..., -0.4290,  0.1884,  0.0928],
        [-0.0908,  0.1647,  0.0419,  ..., -0.2197,  0.0784,  0.2627],
        [-0.3871, -0.0272,  0.0418,  ..., -0.1051,  0.1896,  0.2582],
        ...,
        [-0.2752,  0.1096, -0.0129,  ..., -0.2690,  0.0058,  0.0249],
        [-0.0914,  0.1907,  0.2269,  ..., -0.2555,  0.1743,  0.0336],
        [-0.3719, -0.3967,  0.0296,  ..., -0.0283,  0.2929, -0.2806]])

# Pt5. Indexing in Faiss

In [ ]:
index = faiss.IndexFlatL2(dimension)  # BERT embedding size
faiss.normalize_L2(embeddings.numpy())
index.add(embeddings)
faiss.write_index(index, 'video1.index')

# Pt6. Querying Faiss

In [ ]:
query = "America"
k = 5

encoding = tokenizer.encode_plus(
    query,                 # List of input texts
    padding=True,              # Pad to the maximum sequence length
    truncation=True,           # Truncate to the maximum sequence length if necessary
    return_tensors='pt',       # Return PyTorch tensors
    add_special_tokens=True    # Add special tokens CLS and SEP
)
query_input_ids = encoding['input_ids']  # Token IDs
query_attention_mask = encoding['attention_mask']  # Attention mask

with torch.no_grad():
  outputs = model(query_input_ids, attention_mask=query_attention_mask)
  query_embeddings = outputs.last_hidden_state.mean(dim=1)

distances, ann = index.search(query_embeddings.numpy(), k)
print(distances)
print(ann) #ann is the approximate nearest neighbour
results = pd.DataFrame({'distances': distances[0], 'ann': ann[0]})
sentences[ann[0][0]]
sentences_df = pd.DataFrame(sentences)
merge = pd.merge(results, sentences_df, left_on='ann', right_index=True)
merge



[[66.65237  66.91025  66.935486 67.13924  67.44388 ]]
[[ 0  2 28  6 24]]


,distances,ann,0
0,66.652367,0,"The United States of America, commonly known a..."
1,66.910248,2,It shares land borders with Canada to its nort...
2,66.935486,28,It is one of the world's nuclear-weapon states.
3,67.139236,6,"Beginning in 1607, British colonization led to..."
4,67.443878,24,The American economy accounts for over a quart...
